In [2]:
import cython #ENSURE cython package is installed on computer/canopy
from gensim.models import phrases 
from gensim import corpora, models, similarities #calc all similarities at once, from http://radimrehurek.com/gensim/tut3.html
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from statistics import mean
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
from string import ascii_letters, digits
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.test.utils import datapath
import os
import csv

In [ ]:


def tokenize_folder():
    sentences = []
    for fname in os.listdir(os.path.join(os.getcwd(), 'cTakes archive')): #go through all file names in current directory
        if 'txt' in fname:
            with open(os.path.join(os.getcwd(), 'cTakes archive', fname)) as file:
                for l in file:
                    l = l.translate(translator)
                    l = re.sub(apas_error, "'", l)
                    l = l.strip()
                    l = l.lower()
                    l = stripword(l)
                    if len(l.split()) > 7:
                        l = l + ' .'
                        sentences.append(l.split())
    return sentences


def tokenize_json():
    sentences = []
    with open('train.json') as json_f:
        dataset = json.load(json_f)
        for data in dataset:
            for l in data['supports']:
                l = l.translate(translator)
                l = l.strip()
                l = tokenizer.tokenize(l)
                l = [stripword(n.lower()) for n in l]
                for sent in l:
                    if len(sent.split()) > 7:
                        sent = sent + ' .'
                        sentences.append(sent.split())
    return sentences


def tokenize_json2():
    sentences = []
    with open('train1.0.json') as json_f:
        dataset = json.load(json_f)
        data = dataset[DATA_KEY]
        for datum in data:
            l = datum[DOC_KEY][CONTEXT_KEY]
            l = l.translate(translator)
            l = l.strip()
            l = tokenizer.tokenize(l)
            l = [stripword(n.lower()) for n in l]
            for sent in l:
                if len(sent.split()) > 7:
                    sent = sent + ' .'
                    sentences.append(sent.split())
    return sentences


def tokenize_file():
    sentences = []
    with open(os.path.join(os.getcwd(), 'cTakes archive', fname)) as file:
        for i in range(0,7):
            next(file, None)
        for l in file:
            l = l.translate(translator)
            l = l.strip()
            l = l.lower()
            l = stripword(l)
            sentences.append(l.split())
    return sentences




def tokenize_xml():
    sentences = []
    for fname in os.listdir(os.path.join(os.getcwd(), 'cTakes archive/2try')): #go through all file names in current directory
        if 'xml' in fname:
            tree = ET.fromstring(open(os.path.join(os.getcwd(), 'cTakes archive/2try', fname)).read())
            for node in tree.iter('TEXT'):
                for elem in node.iter():
                    l = elem.text.translate(translator)
                    l = l.strip()
                    l = l.lower()
                    l = tokenizer.tokenize(l)
                    l = [stripword(n) for n in l]
                    for i, sent in enumerate(l):
                        if i < 1:
                            continue
                        elif len(sent.split()) > 7:
                            sent = sent + ' .'
                            sentences.append(sent.split())
    return sentences

In [48]:
def stripword(word):
    word = word.replace("[", "")
    word = word.replace("]", "")
    word = word.replace("'", "")
    word = word.replace("'", "")
    word = word.replace(",", "")
    word = word.replace(".", "")
    word = word.replace("_", "")
    word = word.replace(":", "")
    word = word.replace("-", "")
    word = word.replace("*", "")
    word = word.replace("/", "")
    word = word.replace("(", "")
    word = word.replace(")", "")
    word = word.replace("´", "")
    word = word.replace("`", "")
    word = word.replace(";", "")
    return word

In [28]:
def tokenize():
    translator = str.maketrans(ascii_letters, ascii_letters, digits)
    sentences = []
    for fname in os.listdir(os.path.join(os.getcwd(), 'cTakes archive')): #go through all file names in current directory
        if 'txt' in fname:
            with open(os.path.join(os.getcwd(), 'cTakes archive', fname)) as file:
                for i in range(0,7):
                    next(file, None)
                for l in file:
                    l = l.translate(translator)
                    l = l.strip()
                    l = l.lower()
                    l = stripword(l)
                    sentences.append(l.split())
    return sentences

sentences = tokenize()
print(sentences[5])
print(len(sentences))

['kote', 'oa']
2728214


In [68]:
#bigrams = phrases.Phrases(sentences, min_count=3, delimiter=b' ')
#sentences = [bigram[line] for line in sentences]
#trigram = phrases.Phrases(sentences, min_count=1, delimiter=b' ')

#trigram = phrases.Phraser(trigrams)

#for sent in sentences:
    #bigram = [b for b in bigrams[sent] if b.count(' ') == 1]
    #trigram = [t for t in trigrams[bigram[sent]] if t.count(' ') == 2]

#print(trigram)

In [3]:
bigram_transformer = phrases.Phrases(sentences) 
bigram = phrases.Phraser(bigram_transformer)



trigram = phrases.Phrases(bigram_transformer[sentences])
#trigram = phrases.Phraser(trigram_transformer)

#for sentence in sentences:                
        #print(trigram[bigram[sentence]])

#print(bigram)

In [4]:
currentmodel= Word2Vec(trigram[bigram[sentences]], workers=4, sg=0,size=50, min_count=5,window=5, sample=1e-3)

In [5]:
currentmodel.save("C:/Users/schaa/PycharmProjects/UCSF_NLP/cosine_model/cosine_similarity_metric")

In [3]:
currentmodel= Word2Vec.load("cosine_similarity_metric")

In [5]:
len(currentmodel.wv.vocab) 

200983

In [6]:
currentmodel.wv['wife']

array([-0.5999068 ,  0.35416883, -0.04330867,  0.02633828,  0.39897603,
       -0.19057631, -0.13819359, -0.05821467, -0.01146783, -0.20452908,
        0.05992888, -0.08144449,  0.17725985, -0.22243252,  0.24130572,
        0.8107811 ,  0.01259795,  0.32264927,  0.2964318 ,  0.35275242,
        0.29182708, -0.26796788, -0.03521559, -0.39380762, -0.00874169,
       -0.34323993,  0.0502866 ,  0.04094986, -0.14384526, -0.0987082 ,
        0.33888194,  0.33652577,  0.5721614 ,  0.02995919, -0.27039793,
       -0.31809515,  0.11355288, -0.2792849 , -0.03718785,  0.00680792,
       -0.00730454, -0.33719754, -0.32226714,  0.24901228, -0.43193877,
       -0.44604075,  0.03743329, -0.43035948, -0.01268514, -0.62454164],
      dtype=float32)

In [22]:
currentmodel.wv.most_similar('vodka', topn=10)

[('strawberry', 0.887153685092926),
 ('nonfat', 0.8739331364631653),
 ('cocoa_powder', 0.8732395768165588),
 ('sugarsweetened', 0.8591521382331848),
 ('onion', 0.8565974235534668),
 ('noncaloric', 0.8525886535644531),
 ('wholemeal', 0.8516848683357239),
 ('drinks_containing', 0.850907564163208),
 ('blackcurrant', 0.8494446873664856),
 ('roasted', 0.8492937088012695)]

In [26]:
def cosine_similarity(word1, word2):
    return 1 - spatial.distance.cosine(currentmodel.wv[word1], currentmodel.wv[word2])

In [25]:
def extract_txt(filename):
    all_txt = []
    translator = str.maketrans(ascii_letters, ascii_letters, digits)
    with open(filename) as csvfile:
        csvdata = csv.reader(csvfile)
        next(csvdata, None)
        for row in csvdata:
            row13 = row[13]
            row3 = row[3]
            row13 = row13.translate(translator)
            row13 = row13.strip()
            row13 = row13.lower()
            row13 = stripword(row13)
            row3 = row3.translate(translator)
            row3 = row3.strip()
            row3 = row3.lower()
            row3 = stripword(row3)
            all_txt.append([row13.split(), row3.split()])
    return all_txt


all_txt = extract_txt('combined_procedures.csv')
    

def combined_cosine_similarity(n):
    cosine_accum = 0
    number = 0
    for word1 in all_txt[n][0]:
        for word2 in all_txt[n][1]:
            try:
                cosine_accum += cosine_similarity(word1, word2)
                number += 1
            except:
                cosine_accum += 0
    #for i in all_txt[n][0]:
        #for j in all_txt[n][1]:
            #try:
                #word1 = '_'.join(all_txt[n][0][i:i+2])
                #word2 = '_'.join(all_txt[n][1][j:j+2])
                #cosine_accum += cosine_similarity(all_txt[n][0][i], all_txt[n][1][j])
                #number += 1
            #except:
                #cosine_accum += 0
                
                
    cosine_average = cosine_accum / number
    for word1 in all_txt[n][0]:
        print(word1)
    for word2 in all_txt[n][1]:
        print(word2)
    for i in range(0, len(all_txt[n][0])):
        print('_'.join(all_txt[n][0][i:i+2]))
    for j in range(0, len(all_txt[n][1])):
        print('_'.join(all_txt[n][1][j:j+2]))
    return cosine_average

def combined_cosine_similarity2(n):
    cosine_average = 0
    count = 0
    for m in range(0,len(all_txt[n][0])):
        try:
            word1 = '_'.join(all_txt[n][0][m:m+3])
            word2 = '_'.join(all_txt[n][1][m:m+3])
            cosine_average += cosine_similarity(word1, word2)
            count += 1
        except:
            pass
    if count == 0:
        for m in range(0,len(all_txt[n][0])):
            try:
                word1 = '_'.join(all_txt[n][0][m:m+2])
                word2 = '_'.join(all_txt[n][1][m:m+2])
                cosine_average += cosine_similarity(word1, word2)
                count += 1
            except:
                pass
    if count == 0:
        for word1 in all_txt[n][0]:
            for word2 in all_txt[n][1]:
                try:
                    cosine_average += cosine_similarity(word1, word2)
                    count += 1
                except:
                    pass
    print('_'.join(all_txt[n][0]))
    print('_'.join(all_txt[n][1]))
    if count == 0:
        return 0
    else:
        return cosine_average/count
    
for n in range(0,100):
    print(combined_cosine_similarity2(n))
    print()

NameError: name 'stripword' is not defined

In [24]:
difference_vector = currentmodel.wv['staphylococcal_infections'] - currentmodel.wv['staph']
print(difference_vector)
print(combined_cosine_similarity2(93))

[ 0.12020156 -0.30863494 -0.20496017 -0.05184628 -0.17546077  0.14887759
 -0.21460165 -0.14561042  0.13152403  0.00774099 -0.0890884   0.17240329
  0.15677413 -0.00983274  0.12491821  0.06888674  0.1877844   0.23370658
  0.17949247 -0.10404465 -0.08324734 -0.1560629   0.02284857  0.06156752
  0.03519148 -0.04163155 -0.00911897  0.06682156  0.03250984  0.00344628
 -0.19634135 -0.11050756  0.10098938  0.09158704  0.157167   -0.11486799
 -0.11026829 -0.0537328   0.08850987  0.05218657 -0.08690442  0.14819366
  0.30944616 -0.03633654 -0.21226013  0.12385279 -0.00097142 -0.08887728
  0.0700753  -0.10910447]


NameError: name 'combined_cosine_similarity2' is not defined